<a href="https://colab.research.google.com/github/jhermosillo/diplomado_CDD2019/blob/master/04%20Clasificacion_Agrupamiento/2%20Modelos%20Probabilistas/Notebooks/Naive_Bayes_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-size:50px;" align="left"> <img align="left" width="400" src="../img/cerebro_1.jpg"/>Clasificador Naïve Bayes<br>
    Detector de Spam

In [1]:
import pandas as pd
import numpy as np

import math

## Archivo de datos preprocesados

### Matriz BOW

* Los correos vienen etiquetados de la siguiente forma: `correo_enron1_<0/1>_<#email>` donde $0\equiv Ham$ y $1\equiv Spam$.

In [3]:
data = pd.read_csv('../Data/frequencia_palabras.csv') # lectura del documento con pandas
# visualizacion
print(data.shape)
data.head()

(9406, 1002)


,Unnamed: 0,WORD,correo_enron1_0_0,correo_enron1_0_1,correo_enron1_0_2,correo_enron1_0_3,correo_enron1_0_4,correo_enron1_0_5,correo_enron1_0_6,correo_enron1_0_7,...,correo_enron1_1_490,correo_enron1_1_491,correo_enron1_1_492,correo_enron1_1_493,correo_enron1_1_494,correo_enron1_1_495,correo_enron1_1_496,correo_enron1_1_497,correo_enron1_1_498,correo_enron1_1_499
0,0,fw,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,everyone,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,access,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,change,1,0,0,0,5,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,4,security,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


La siguiente función separa los datos para prueba y entrenamiento, el argumento 'resto' indica la cantidad de correos que se desea para los datos de prueba. <br>
La salida son dos diccionarios, uno de entrenamiento y el otro para prueba del modelo.

In [4]:
def datosEP(data, resto):
    conjunto1 = {'WORD':data['WORD']}
    n_instPN = 500 - resto
    for i in range(2):
        for j in range(n_instPN):
            llave = 'correo_enron1_'+str(i)+"_"+str(j)
            conjunto1[llave] = data[llave]
    conjunto2 = {'WORD':data['WORD']}
    for i in range(2):
        for j in range(resto):
            llave = 'correo_enron1_'+str(i)+"_"+str(j)
            conjunto2[llave] = data[llave]
    return [conjunto1,conjunto2]

In [5]:
X_train, X_test = datosEP(data,100)
print(len(X_train.keys()),len(X_test.keys()))

801 201


In [9]:
pd.DataFrame(X_train).head()

,WORD,correo_enron1_0_0,correo_enron1_0_1,correo_enron1_0_2,correo_enron1_0_3,correo_enron1_0_4,correo_enron1_0_5,correo_enron1_0_6,correo_enron1_0_7,correo_enron1_0_8,...,correo_enron1_1_390,correo_enron1_1_391,correo_enron1_1_392,correo_enron1_1_393,correo_enron1_1_394,correo_enron1_1_395,correo_enron1_1_396,correo_enron1_1_397,correo_enron1_1_398,correo_enron1_1_399
0,fw,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,everyone,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,access,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,change,1,0,0,0,5,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,security,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Entrenamiento
La distribución de probabilidad de la Variable Aleatoria $\textrm{x}$ es una distribución multinomial (conteos de palabras) de la conjunción de palabras únicas (vocabulario-$Voc$).<br>

### &#9998; Entrena al modelo utilizando la siguiente fórmula:

$$ P(x_i|y_j) = \frac{n_i + 1}{|S_j| + k} $$ como se vió en clase.

* $|S_j|$ es la suma de los conteos de cada palabra para la clase $y_j$. 
* $k=|Voc|$.
* $n_i$ es la suma de los conteos para la palabra $i$.

&#9758; **Crea una función que reciba un diccionario y devuelva dos vectores: theta_Positivo y theta_Negativo.**

In [8]:
def fit(X_train):
    k = len(X_train['WORD']) # numero de palabras
    print('numero de palabras: ',k)
    datos_p = int((int(len(X_train.keys()))-1)/2)
    print("datos_p: ", datos_p)
    
    theta_p = np.ones(k)
    theta_n = np.ones(k)
    
    indice_p = 0 # indice para los correos ham
    indice_n = 1 # indice para los correos spam
    
    #entranamiento para theta positivo => S_p
    S_p = 0 # total de conteos de las instancias ham
    """ TU CODIGO AQUI"""
    
    """ ------------- """
    print(theta_p.shape)
    
    #entranamiento para theta negativo => S_n
    S_n = 0 # total de conteos de las instancias ham
    """ TU CODIGO AQUI"""
    
    """ ------------- """
    print(theta_n.shape)
    return [theta_p,theta_n]
        

&#9758; **Prueba el entrenamiento**

In [12]:
theta_p, theta_n = fit(X_train)

numero de palabras:  9406
datos_p:  400
|S_p| =  24528
(9406,)
|S_p| =  31518
(9406,) (9406,)


In [13]:
print("La suma para vector debe ser 1 \n Theta_p : ",np.sum(theta_p))
print("La suma para vector debe ser 1 \n Theta_n : ",np.sum(theta_n))

La suma para vector debe ser 1 
 Theta_p :  1.0
La suma para vector debe ser 1 
 Theta_n :  1.0


### Prueba del Modelo
Como se tiene una distribución miltinomial, se utilizará la formula de tal distribución para la predicción de una instancia $\textrm{x}$ dado theta_positivo y theta_negativo.<br> <br>

$$ p(\textrm{x}|\theta_Pos) =  \prod_{i=1}^{k} p(x_i|\theta_Pos) = n!*\frac{\theta_{1}^{x_1}}{x_1!}*...*\frac{\theta_k^{x_k}}{x_k!}$$ <br>

$$ p(\textrm{x}|\theta_Neg) =  \prod_{i=1}^{k} p(x_i|\theta_Neg) = n!*\frac{\theta_{1}^{x_1}}{x_1!}*...*\frac{\theta_k^{x_k}}{x_k!}$$ <br>

$$ LR = \frac{p(\textrm{x}|\theta_Pos)}{p(\textrm{x}|\theta_Neg)}$$ <br>

Este modelo utiliza la razón de verosimilitud (LR) para hacer la predición.<br>
Si LR > 1 -> y=1 <br>
Si LR < 1 -> y=0 <br>

In [14]:
def LR(x,theta_p, theta_n):
    """
    LR=P(x|y=+)/P(x|y=-)
    """
    lr=None
    """ TU CODIGO AQUI """
    
    """ -------------  """
    
    return lr

In [15]:
"""
    Funcion que tranforma un diccionario a un arreglo de numpy, sin tomar en cuenta la primera llave del diccionario
"""
def diccionarioArreglo(X_test):
    listaPrin = []
    for llave in X_test.keys():
        if llave!="WORD": # escapar las palbras
            listaPrin.append([i for i in X_test[llave]])
    return np.array(listaPrin).T        

La siguiente función separa los datos para prueba y entrenamiento, el argumento 'procentaje' indica el procentaje de los datos de prueba que se desea. <br>
La salida son dos arreglos de numpy, uno de entranamiento y el otro para prueba del modelo y su respetivo vector de clases.

In [16]:
def datosEntPrue(data, porcentaje):
    tamTotal = len(data.keys())-2 # no se cuenta la columna de palabras en el diccionario
    tamPrueba = int((porcentaje*tamTotal)/100)
    tamEntrenamiento = tamTotal - tamPrueba
    print("tamaño del conjunto de (prueba,entrenamiento) : ", tamPrueba,",",tamEntrenamiento)
    
    y_train = np.array([0 for i in range(int(tamEntrenamiento/2))]+[1 for i in range(int(tamEntrenamiento/2))])
    conjunto1 = {'WORD':data['WORD']}
    for i in range(2):
        for j in range(int(tamEntrenamiento/2)):
            llave = 'correo_enron1_'+str(i)+"_"+str(j)
            conjunto1[llave] = data[llave]
            
    y_test = np.array([0 for i in range(int(tamPrueba/2))]+[1 for i in range(int(tamPrueba/2))])
    conjunto2 = {'WORD':data['WORD']}
    for i in range(2):
        for j in range(int(tamPrueba/2)):
            llave = 'correo_enron1_'+str(i)+"_"+str(j)
            conjunto2[llave] = data[llave]
    return [diccionarioArreglo(conjunto1), y_train,diccionarioArreglo(conjunto2),y_test]

In [17]:
x_train,y_train,x_test,y_test = datosEntPrue(data, 20)
print("Dimension de el conjunto de entrenamiento: ",x_train.shape," con su respectivo vector de etiquetas y: ",y_train.shape)
print("Dimension de el conjunto de prueba: ",x_test.shape," con su respectivo vector de etiquetas y: ",y_test.shape)

tamaño del conjunto de (prueba,entrenamiento) :  200 , 800
Dimension de el conjunto de entrenamiento:  (9406, 800)  con su respectivo vector de etiquetas y:  (800,)
Dimension de el conjunto de prueba:  (9406, 200)  con su respectivo vector de etiquetas y:  (200,)


In [18]:
print("Dimension de los conjuntos de entranamiento y prueba: ",x_train.shape,x_test.shape)

Dimension de los conjuntos de entranamiento y prueba:  (9406, 800) (9406, 200)


## Test del modelo
La siguiente función realiza el test del conjunto de pruebas.<br>
Recibe el arreglo de datos de prueba, el vector theta positivo y negativo para el LR.

In [19]:
def test(X_test,theta_p, theta_n):
    tamEjemplos = x_test.shape[1]
    y = np.zeros(tamEjemplos)
    print(y.shape)
    for i in range(tamEjemplos):
        #print("si")
        try:
            #print("eel",P(X_test[:,i],theta_p, theta_n))
            y[i] = P(X_test[:,i],theta_p, theta_n)
        except:
            y[i] = 1
    return y

In [20]:
y_predic = test(x_test,theta_p, theta_n)
print(y_predic)

(200,)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]


## Matriz de confusión

In [21]:
def matrizCnf(y_test,y_predic):
    """
        hay que recordar que por la construccion del conjunto de prueba y entrenamiento,
        la primera mitad de los ejemplos (entrenamiento, prueba) son positivos (ham) y 
        el resto son negativos (spam)
    """
    totalInstancias = y_test.shape[0]
    tPositivo = len(np.where(y_predic[0:100]==y_test[0:100])[0]) # true positive Los primeros 100 son correos ham
    fPositivo = len(np.where(y_predic[0:100]!=y_test[0:100])[0]) # false positive Los primeros 100 son correos ham
    #print(tPositivo,fPositivo)
    tNegativo = len(np.where(y_predic[100:200]==y_test[100:200])[0]) # true negative Los primeros 100 son correos spam
    fNegativo = len(np.where(y_predic[100:200]!=y_test[100:200])[0]) # false negative Los primeros 100 son correos spam
    #print(tNegativo,fNegativo)
    matriz = {'1':["Actual: positivo","Actual: negativo"]}
    matriz["prediccion: positivo"] = [tPositivo,fPositivo]
    matriz["prediccion: negativo"] = [fNegativo,tNegativo]
    return matriz

In [22]:
matrizConfusion = matrizCnf(y_test,y_predic)

In [ ]:
pd.DataFrame(matrizConfusion).head()